In [1]:
import collections
from functools import reduce


import numpy as np
import pandas as pd

In [2]:
statins =[
    'MED1140861958',
    'MED1140888594',
    'MED1140888648',
    'MED1141146234',
    'MED1141192410',
    'MED1141146138',
    'MED1140861922'    
]

In [3]:
statins_df = dict()
for i in statins:
    print(i)
    statins_df[i] = pd.read_csv(
        'statin_phes/{}.phe'.format(i), sep='\t',
        low_memory=False,
        usecols=[1,2]
    )
    statins_df[i]['IID'] = statins_df[i]['IID'].map(lambda x: str(x))
    statins_df[i][i] = statins_df[i][i].map(lambda x: int(x))

MED1140861958
MED1140888594
MED1140888648
MED1141146234
MED1141192410
MED1141146138
MED1140861922


In [4]:
merged_df = statins_df[statins[0]].merge(
    statins_df[statins[1]], on='IID', how='left'
).merge(
    statins_df[statins[2]], on='IID', how='left'
).merge(
    statins_df[statins[3]], on='IID', how='left'
).merge(
    statins_df[statins[4]], on='IID', how='left'
).merge(
    statins_df[statins[5]], on='IID', how='left'
).merge(
    statins_df[statins[6]], on='IID', how='left'
).fillna(-9)

In [5]:
for i in statins:
    merged_df[i] = merged_df[i].map(lambda x: int(x))

In [6]:
merged_df = merged_df[merged_df['IID'] != 'FID']

In [7]:
def aggregate_phe(p0, p1):
    case    = ((p0 == 2)  | (p1 == 2))
    missing = ((p0 == -9) & (p1 == -9))
    control = np.invert((case | missing))
    return (np.where(case, 2, 0) + np.where(control, 1, 0) + np.where(missing, -9, 0))


In [11]:
phe_mat = merged_df[statins].values

In [12]:
phe_vec = aggregate_phe(
    aggregate_phe(
        aggregate_phe(
            phe_mat[:, 0], 
            phe_mat[:, 1]
        ),
        aggregate_phe(
            phe_mat[:, 2], 
            phe_mat[:, 3]
        ),        
    ), 
    aggregate_phe(
        aggregate_phe(
            phe_mat[:, 4], 
            phe_mat[:, 5]
        ),
        phe_mat[:, 6], 
    ), 
)

In [13]:
collections.Counter(list(phe_vec))

Counter({1: 416478, -9: 14262, 2: 86144})

In [16]:
df = pd.DataFrame(collections.OrderedDict([
    ('#FID', merged_df['IID']),
    ('IID',  merged_df['IID']),
    ('statin', phe_vec)
]))

In [17]:
df.to_csv('statin_phes/statin.phe', sep='\t', index=False)